
Nome: Adriano Ulrich do Prado Wiedmann
---

Matrícula: 202014824

In [29]:
#allows the drive to be accessed
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [30]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# tells matplotlib to embed plots within the notebook
%matplotlib inline

# Pytorch
import torch

# modules to adequate the NN
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

# modules to process strings
from string import punctuation

# module to organize the words
from collections import Counter

# Measuring execution time
import time


In [31]:
def testar_gpu():
	train_on_gpu = torch.cuda.is_available() #Observa se a GPU está disponivel
	if train_on_gpu: #Se sim
		device = torch.device('cuda') #Seleciona o device como GPU
		print("Treinando na GPU") #E manda a mensagem
	else: #Se não
		device = torch.device('cpu') #Seleciona o device como cpu
		print("GPU indisponível, treinando na CPU") #E avisa que a GPU não esta disponível
	return device

device = testar_gpu()

Treinando na GPU


In [32]:
# read data from text files
with open('/content/drive/MyDrive/DP6/data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('/content/drive/MyDrive/DP6/data/labels.txt', 'r') as f:
    labels = f.read()

In [33]:
print(reviews[:1000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [34]:
# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

In [35]:
# split by new lines and spaces
reviews_split = all_text.split('\n')

print(f'O número de reviews presentes no dataset é:{len(reviews_split)}.')

O número de reviews presentes no dataset é:25001.


In [36]:
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

print(f'O número de palavras presentes no dataset é:{len(words)}.')

O número de palavras presentes no dataset é:6020196.


In [37]:
## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

In [38]:
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [39]:
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

In [40]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [41]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


In [42]:
def pad_features(reviews_ints, seq_length):
    """
    Retorna os features de 'review_ints', em que cada review é preenchida com
    zeros ou truncada no tamanho determinado pela entrada 'seq_length'.

    Parâmetros
    ----------
    reviews_ints: list
      Lista que contém todas as avaliações (mapeadas em números inteiros), uma
      em cada elemento da lista.

    seq_length: int
      Valor que será usado como tamanho padrão de cada linha da matriz
      'features'.

    Retorna
    --------
    features: np.array
      Matriz que contém na linha 'i' o vetor de features do exemplo 'i'. O
      número de colunas é igual a 'seq_length'.

    Instruções
    ------------
    Para cada vetor contido em 'reviews_ints', adicione zeros caso o tamanho
    do vetor seja menor que 'seq_length' ou trunque o vetor em 'seq_length'
    caso o tamanho do vetor seja maior.
    """

    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]

    return features

In [43]:
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

In [44]:
split_frac = 0.85

## split data into training and test data (features and labels, x and y)
split_idx = int(len(features)*split_frac)
train_x, test_x = features[:split_idx], features[split_idx:]
train_y, test_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(21250, 200) 
Test set: 		(3750, 200)


In [45]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [46]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print()
print('Sample label size: ', sample_y.size()) # batch_size

Sample input size:  torch.Size([50, 200])

Sample label size:  torch.Size([50])


In [47]:
class SentimentRNN(nn.Module):
    """
    Classe referente ao modelo que será usado para realizar a análise de
    sentimento.

    Instruções
    ------------
    Defina as camadas da rede conforme a arquitetura pedida (camada de
    embedding, camada LSTM e camada fully connected com sigmoid no fim).
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):

        # Initialize the model by setting up the layers.

        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # ============================== YOUR CODE HERE ================================
        # Camada de embedding:
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Camada LSTM:
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)

        # Camada dropout
        self.dropout = nn.Dropout(0.3)

        # Camada fully connected:
        self.fc = nn.Linear(hidden_dim, output_size)

        # camada Sigmoid
        self.sig = nn.Sigmoid()
        # ==============================================================================

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out[:, -1, :] # getting the last time step output

        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))

        return hidden


In [48]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

model = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers).to(device)

print(model)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [49]:
lr=0.001
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
clip = 5

In [50]:
"""
Treine os parâmetros das camadas de embedding e LSTM da rede contida em
'model' com os exemplos contidos em 'data_train' (organizados por
'train_loader'). Em seguida, teste a acurácia e o erro de classificação da
rede com os exemplos contidos em 'data_test' (organizados por
'loader_test').

Instruções
-------------
Efetue o treinamento e testa da rede, realizando uma passagem dos datasets
de treino e de teste a cada época, calculando o erro de classificação de
cada 'batch' e atualizando os parâmetros da rede com o otimizador definido
anteriormente, e então calcule o erro médio de classificação pela rede dos
exemplos de treino (guardando o resultado em 'train_loss'), o error médio
de classificação pela rede dos exemplos de teste (guardando o resultado em
'test_loss') e a acurácia de classificação pela rede dos exemplos de teste
(guardando o resultado em 'acc').
Nota
-------------
O aluno pode fazer uso de quaisquer ferramentas fornecidas pelo Pytorch
para implementar esta função.

É importante lembrar de utilizar a GPU para efetuar quaisquer processamentos
possíveis (utilize a função 'to(device)') a fim de otimizar o tempo de
execução.
"""

START = time.time()

list_train_loss = []
list_test_loss = []
list_acc = []
time1epoch = 0

epochs = 5

for epoch in range(1, epochs+1):
  start = time.time()

  train_loss, test_loss, acc = 0,0,0

  # ============================== YOUR CODE HERE ================================
  model.train()
  hidden = model.init_hidden(batch_size)
  loss = 0
  for inputs_treino, labels_treino in train_loader:
    inputs_treino, labels_treino = inputs_treino.to(device), labels_treino.to(device)
    hidden = tuple([each.data for each in hidden])
    model.zero_grad()
    output, hidden = model(inputs_treino, hidden)
    test_loss = criterion(output.squeeze(), labels_treino.float())
    test_loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), clip).to(device)
    optimizer.step()

  train_loss += test_loss.item()
  model.eval()
  total = 0
  correct = 0
  test_loss = 0
  hidden = model.init_hidden(batch_size)
  for inputs_teste, labels_teste in test_loader:
    inputs_teste, labels_teste = inputs_teste.to(device), labels_teste.to(device)
    output, hidden = model(inputs_teste, hidden)
    test_loss += criterion(output.squeeze(), labels_teste.float())
    pred = torch.round(output.squeeze())
    correct += (pred == labels_teste).sum().item()
    total += batch_size

  test_loss = test_loss/batch_size
  acc = (correct/total)*100


 	# ==============================================================================

  end = time.time()

  list_train_loss.append(train_loss), list_test_loss.append(test_loss), list_acc.append(acc)

  Time = end - start
  if epoch == 1:
    time1epoch = Time

  print('Epoch: ', epoch ,' loss: {:.4f}'.format(test_loss.item()), ' Accuracy: {:.2f}'.format(acc), ' Time spent this epoch: {:.2f}'.format(Time), 'seconds.')

END = time.time()

print()
TIME = (END - START) - time1epoch
print('\n Time spent during training, excluding first epoch: {:.2f}'.format(TIME), 'seconds.')

Epoch:  1  loss: 0.7974  Accuracy: 73.95  Time spent this epoch: 22.71 seconds.
Epoch:  2  loss: 0.6629  Accuracy: 78.13  Time spent this epoch: 22.56 seconds.
Epoch:  3  loss: 0.6722  Accuracy: 81.28  Time spent this epoch: 22.13 seconds.
Epoch:  4  loss: 0.7510  Accuracy: 80.96  Time spent this epoch: 22.41 seconds.
Epoch:  5  loss: 0.8863  Accuracy: 81.17  Time spent this epoch: 22.53 seconds.


 Time spent during training, excluding first epoch: 89.64 seconds.


In [51]:
def pre_processing(test_review):

    """
    Pré-processa a string de entrada 'test_review' de forma a adequá-la como
    parâmetro de entrada para a rede.

    Parâmetros
    ----------
    test_review: str
      String referente ao comentário que se deseja analisar o sentimento pela
      rede neural.

    Retorna
    --------
    features: torch.tensor
      Tensor que contém na linha o vetor de features do exemplo em questão. O
      tamanho do tensor é igual a 'seq_length'.

    Instruções
    ------------
    Realize as mesmas operações que foram feitas com o dataset de treino: i)
    colocar todo o texto em letras minúsculas; ii) excluir pontuação; iii)
    troque as palavras por índices do dicionário 'vocab_to_int'; iv) trunque
    ou adicione zeros ao vetor para que seu tamanho seja igual a 'seq_length'
    (pode ser feito chamando a função pad_features); e v) passe o tensor para a
    GPU ('x.to(device)').
    """

  # ============================== YOUR CODE HERE ================================
    test_review = test_review.lower()
    all_text = ''.join([c for c in test_review if c not in punctuation])

    feature_tensor = [vocab_to_int.get(word, 0) for word in all_text.split()]
    feature_tensor = pad_features([feature_tensor], seq_length=seq_length)
    feature_tensor = torch.tensor(feature_tensor)

    #feature_tensor = feature_tensor.cpu()
    #feature_tensor = feature_tensor.to(device)
 	# ==============================================================================

    return feature_tensor

# test code and generate tokenized review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'
feature_tensor = pre_processing(test_review_neg)
print(feature_tensor.size())

torch.Size([1, 200])


In [52]:
def predict(test_review):

    model.eval()

    """
    Recebe uma string que será pré-processada e então analisada pela rede
    neural. Com base no valor de saída da rede, define um limiar de
    confiabilidade da predição, classificando o comentário como "Positivo",
    "Negativo" ou "Inconclusivo".

    Parâmetros
    ----------
    test_review: str
      String referente ao comentário que se deseja analisar o sentimento pela
      rede neural.

    Instruções
    ------------
    Pré-processe a string 'test_review', chamando a função 'pre_processing()', e
    então passe o vetor pré-processado como entrada à rede neural. Analise o
    valor de saída da rede e printe na tela "Positivo", "Negativo" ou
    "Inconclusivo" com base nos limiares definidos.
    """

    # ============================== YOUR CODE HERE ================================
    processed_review = pre_processing(test_review) # String pré-processada
    features = pad_features(processed_review, seq_length=seq_length)

    batch_size = processed_review.size(0)
    hidden = model.init_hidden(batch_size)
    model.embedding.weight.data = model.embedding.weight.data.to(device)

    processed_review = processed_review.to(device)
    output, hidden = model(processed_review, hidden)

    # Predição binária
    pred = torch.round(output.squeeze())

    output = output.item()
    if output <= 0.25:
      print("Negativo")
    elif output >= 0.75:
      print("Positivo")
    else:
      print("Inconclusivo")
 	  # ==============================================================================


In [53]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'
predict(test_review_neg)
print()

# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'
predict(test_review_pos)
print()

# uncorrelated sentence
test_review_uncorrelated = 'Curso TopEng - 2/2023.'
predict(test_review_uncorrelated)
print()

# review in another language
test_review_another_lenguage = 'O filme é incrível. O melhor filme que já assisti.'
predict(test_review_another_lenguage)

Negativo

Positivo

Inconclusivo

Inconclusivo
